In [1]:
using Pkg; Pkg.activate("..")

using PyPlot, Printf, Statistics, OceanTurb, Dao, JLD2,
        ColumnModelOptimizationProject, ColumnModelOptimizationProject.KPPOptimization

In [2]:
model_N = 30
model_dt = 10*minute
initial_data = 1 
target_data = (4, 7, 10) 
case = "unstable_weak"

datadir = joinpath("..", "data", "perfect_model_experiment")
filepath = joinpath(datadir, case * ".jld2")
data = ColumnData(filepath; initial=initial_data, targets=target_data)
model = KPPOptimization.ColumnModel(data, model_dt, N=model_N)

# Pick some parameters
defaults = DefaultFreeParameters(BasicParameters)

@show propertynames(defaults)

propertynames(defaults) = (:CRi, :CKE, :CNL, :Cτ, :Cstab, :Cunst, :Cb_U, :Cb_T, :Cd_U, :Cd_T)


(:CRi, :CKE, :CNL, :Cτ, :Cstab, :Cunst, :Cb_U, :Cb_T, :Cd_U, :Cd_T)

In [3]:
# Obtain an estimate of the relative error in the temperature and velocity fields
test_nll_temperature = NegativeLogLikelihood(model, data, temperature_loss)
test_nll_velocity = NegativeLogLikelihood(model, data, velocity_loss)
test_link_temperature = MarkovLink(test_nll_temperature, defaults)
test_link_velocity = MarkovLink(test_nll_velocity, defaults)

error_ratio = test_link_velocity.error / test_link_temperature.error
@printf "Ratio of velocity error to temperature error: %.4f\n" error_ratio 

# Build the weighted NLL, normalizing temperature error relative to velocity error.
weights = (1, 1, 10*round(Int, error_ratio), 0)
nll = NegativeLogLikelihood(model, data, weighted_fields_loss, weights=weights);

Ratio of velocity error to temperature error: 6.2906


In [4]:
# Obtain the first link in the Markov chain
first_link = MarkovLink(nll, defaults)
@show nll.scale = first_link.error * 0.5 

# Use a non-negative normal perturbation
std = DefaultStdFreeParameters(0.05, typeof(defaults))
sampler = MetropolisSampler(NormalPerturbation(std))

ninit = 10^2
chain = MarkovChain(ninit, first_link, nll, sampler)
@show chain.acceptance 

nll.scale = first_link.error * 0.5 = 0.03447890171006672
chain.acceptance = 0.86


0.86

In [5]:
dsave = 10^3
chainname = "test_markov_chain"
chainpath = "$chainname.jld2"
@save chainpath chain

tstart = time()
for i = 1:4
    tint = @elapsed extend!(chain, dsave)
    println(status(chain))

    @sprintf("Tc: %.2f seconds. Elapsed wall time: %.4f minutes.", tint, (time() - tstart)/60)
    println("Optimal parameters:")
    @show chain[1]
    @show optimal(chain)
    @show chain[end]

    oldchainpath = chainname * "_old.jld2"
    mv(chainpath, oldchainpath, force=true)
    @save chainpath chain
    rm(oldchainpath)
end

               length | 1100
           acceptance | 0.797272727
 initial scaled error | 2.000000000
 optimal scaled error | 0.936899762

Optimal parameters:
chain[1] = MarkovLink{Float64,BasicParameters{Float64}}([0.3, 4.32, 6.33, 0.4, 2.0, 6.4, 0.599, 1.36, 0.5, 2.5], 0.06895780342013344)
optimal(chain) = MarkovLink{Float64,BasicParameters{Float64}}([0.167128, 11.8146, 7.05515, 0.387237, 0.998389, 15.2533, 0.642754, 5.08637, 0.134183, 2.5258], 0.032303274816019766)
chain[end] = MarkovLink{Float64,BasicParameters{Float64}}([0.176989, 10.399, 7.02167, 0.492112, 0.288582, 12.1095, 0.9576, 4.1824, 0.164858, 3.46927], 0.060222350565814604)
               length | 2100
           acceptance | 0.825714286
 initial scaled error | 2.000000000
 optimal scaled error | 0.884649359

Optimal parameters:
chain[1] = MarkovLink{Float64,BasicParameters{Float64}}([0.3, 4.32, 6.33, 0.4, 2.0, 6.4, 0.599, 1.36, 0.5, 2.5], 0.06895780342013344)
optimal(chain) = MarkovLink{Float64,BasicParameters{Float64}}([